In [1]:
import os
import argparse
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import csv

# Useful if you want to perform stemming.
import nltk
stemmer = nltk.stem.PorterStemmer()

categories_file_name = r'/workspace/datasets/product_data/categories/categories_0001_abcat0010000_to_pcmcat99300050000.xml'

queries_file_name = r'/workspace/datasets/train.csv'
output_file_name = r'/workspace/datasets/labeled_query_data.txt'



In [2]:
# parser = argparse.ArgumentParser(description='Process arguments.')
# general = parser.add_argument_group("general")
# general.add_argument("--min_queries", default=1,  help="The minimum number of queries per category label (default is 1)")
# general.add_argument("--output", default=output_file_name, help="the file to output to")

# args = parser.parse_args()
# output_file_name = args.output

# if args.min_queries:
#     min_queries = int(args.min_queries)

# The root category, named Best Buy with id cat00000, doesn't have a parent.

min_queries = 10
root_category_id = 'cat00000'

tree = ET.parse(categories_file_name)
root = tree.getroot()

# Parse the category XML file to map each category id to its parent category id in a dataframe.
categories = []
parents = []


In [3]:
for child in root:
    id = child.find('id').text
    cat_path = child.find('path')
    cat_path_ids = [cat.find('id').text for cat in cat_path]
    leaf_id = cat_path_ids[-1]
    if leaf_id != root_category_id:
        categories.append(leaf_id)
        parents.append(cat_path_ids[-2])
parents_df = pd.DataFrame(list(zip(categories, parents)), columns =['category', 'parent'])

# Read the training data into pandas, only keeping queries with non-root categories in our category tree.
df = pd.read_csv(queries_file_name)[['category', 'query']]
df = df[df['category'].isin(categories)]

In [4]:
category_value_counts= pd.DataFrame(df['category'].value_counts().reset_index().\
                                    rename(columns = {"index": "category", "category": "category_count"}))
faulty_categories = list(category_value_counts[category_value_counts['category_count'] < min_queries]['category'])

while len(faulty_categories) > 0:
    df.loc[df['category'].isin(faulty_categories), 'category'] = df['category'].\
    map(parents_df.set_index('category')['parent'])
    category_value_counts= pd.DataFrame(df['category'].value_counts().reset_index().\
                                    rename(columns = {"index": "category", "category": "category_count"}))
    faulty_categories = list(category_value_counts[category_value_counts['category_count'] < min_queries]['category'])
    

In [4]:
# find faulty categories

category_value_counts= pd.DataFrame(df['category'].value_counts().reset_index().\
                                    rename(columns = {"index": "category", "category": "category_count"}))
faulty_categories = list(category_value_counts[category_value_counts['category_count'] < min_queries]['category'])
df.loc[df['category'].isin(faulty_categories), 'category'] = df['category'].map(parents_df.set_index('category')['parent'])

In [5]:
faulty_categories

[]

In [7]:
df.isnull().sum()

category    0
query       0
dtype: int64